In [5]:
#to classfy a sms intp spam and ham using multinominal naives bayes theorem
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
sms = pd.read_table(url, header=None, names=['label', 'message'])

In [6]:
print(sms.shape)
print("\n")
print(sms['label'].value_counts())
sms

(5572, 2)


ham     4825
spam     747
Name: label, dtype: int64


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
# convert label to a numerical variable
sms['label_num']=sms['label'].map({'ham':0,'spam':1})
sms

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [11]:
#cleaning text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus=[]
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')#this is becuse stopwords have not word included by default


for i in range(sms.shape[0]):
  text= re.sub('[^0-9a-zA-Z]',' ',sms['message'][i])
  text=text.lower()
  text=text.split()
  ps=PorterStemmer()
  text=[ps.stem(word) for word in text if word not in set(all_stopwords)]
  text=' '.join(text)
  sms['message'][i]=text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X=sms.message
y=sms.label_num

In [14]:
# split X and y into training and testing sets
# by default, it splits 75% training and 25% test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [15]:
#Vectorizing our dataset
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X_train_dtm = cv.fit_transform(X_train)

#transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm=cv.transform(X_test)

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [16]:
#building and evaluating model
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
# train the model using X_train_dtm 
%time nb.fit(X_train_dtm,y_train)

CPU times: user 4.36 ms, sys: 28 µs, total: 4.39 ms
Wall time: 9.35 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
#make prediction on x_test
y_pred_class = nb.predict(X_test_dtm)

from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred_class))

# print the confusion matrix
print(metrics.confusion_matrix(y_test,y_pred_class))


#Confusion matrix
#[TN FP
#FN TP]


0.9913854989231874
[[1204    4]
 [   8  177]]


In [20]:
# print message text for the false positives (ham incorrectly classified as spam)

X_test[y_pred_class > y_test]

4937                             k k congratul
4419                             get free call
694     purchas stuff today mail po box number
3415                            pic pleas send
Name: message, dtype: object

In [21]:
# print message text for the false negatives (spam incorrectly classified as ham)

X_test[y_pred_class < y_test]

3132    lookatm thank purchas video clip lookatm charg...
3530    xma new year eve ticket sale club day 10am til...
1893    call 09090900040 listen extrem dirti live chat...
4298    thesmszon com let send free anonym mask messag...
4949    hi ami send free phone number coupl day give a...
2821    interflora not late order interflora flower ch...
2247    hi ya babe x u 4goten bout scammer get smart t...
4514                       money wine number 946 wot next
Name: message, dtype: object

In [22]:
# calculate AUC
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test, y_pred_prob)

#AUC is useful as a single number summary of classifier performance
#Higher value = better classifier

0.9908313943082155